In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from pathlib import Path
from sklearn.utils import compute_class_weight
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from keras.callbacks import ModelCheckpoint
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
X_attacker = np.load(Path("CSVsPred/np_attackers_pred.npy")) #ClosestCarCSVsModel2/np_attacker.npy
X_normal = np.load(Path("CSVsPred/np_normal_pred.npy"))
X_normal_split, p = train_test_split(X_normal, test_size=0.70, random_state=42, shuffle=True)
print(X_normal_split.shape)
print(X_attacker.shape)

(4293, 100, 16)
(8164, 100, 16)


In [4]:
y = np.asarray([0]*X_normal_split.shape[0] + [1]*X_attacker.shape[0]).astype('float32')
X = np.concatenate((X_normal_split, X_attacker), axis=0)
print(X.shape)
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, shuffle=True)

(12457, 100, 16)
(12457,)


In [6]:
n_feats = 16
N = 64
model = Sequential()
model.add(Conv2D(N, (3, 3), activation='relu', input_shape=(X_train.shape[1], n_feats, 1), padding="same"))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(N*2, (3, 3), activation='relu', padding="same"))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(N*4, (3, 3), activation='relu', padding="same"))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 16, 64)       640       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 50, 8, 64)        0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 50, 8, 128)        73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 25, 4, 128)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 25, 4, 256)        295168    
                                                                 
 flatten (Flatten)           (None, 25600)            

In [ ]:
n_feats = 16
# X_train = np.expand_dims(X_train, axis=-1)
# X_test = np.expand_dims(X_test, axis=-1)

N = 64
model = Sequential()
model.add(Conv2D(N, (3, 3), activation='relu', input_shape=(X_train.shape[1], n_feats, 1), padding="same"))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(N*2, (3, 3), activation='relu', padding="same"))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(N*4, (3, 3), activation='relu', padding="same"))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# model = make_model(input_shape=(100, 12, 1), num_classes=2)

# compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


mc = ModelCheckpoint('best_model2_V2.h5', monitor='val_loss', mode='min', save_best_only=True)


class_weights = compute_class_weight(class_weight = "balanced", classes= np.unique(y_train), y= y_train)
class_weights = dict(zip(np.unique(y_train), class_weights))
    
model.fit(X_train, y_train, 
          epochs=300, 
          batch_size=9000, 
          class_weight=class_weights,
          shuffle=True,
          validation_split=0.1,
          callbacks=[mc])

model.save('model2_V2.h5')
y_pred = model.predict(X_test, batch_size=2000)
y_pred[y_pred > 0.5] = 1
y_pred[y_pred <= 0.5] = 0
y_pred.flatten()

print('Accuracy:', accuracy_score(y_test, y_pred.flatten()))
print('Precision:', precision_score(y_test, y_pred.flatten()))

In [ ]:
model.fit(X_train, y_train, 
          epochs=300, 
          batch_size=9000, 
          class_weight=class_weights,
          shuffle=True,
          validation_split=0.1,
          callbacks=[mc])

model.save('model2_V2.h5')
y_pred = model.predict(X_test, batch_size=2000)
y_pred[y_pred > 0.5] = 1
y_pred[y_pred <= 0.5] = 0
y_pred.flatten()

print('Accuracy:', accuracy_score(y_test, y_pred.flatten()))
print('Precision:', precision_score(y_test, y_pred.flatten()))

In [ ]:
n_feats = 16

# define the model
model = Sequential()
model.add(Bidirectional(LSTM(64, input_shape=(None, n_feats)))) # "None" allows for varying input lengths
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
class_weights = compute_class_weight(class_weight = "balanced", classes= np.unique(y_train), y= y_train)
class_weights = dict(zip(np.unique(y_train), class_weights))

mc = ModelCheckpoint('best_model2.h5', monitor='val_loss', mode='min', save_best_only=True)

model.fit(X_train, y_train, 
          epochs=70, 
          batch_size=9000, 
          class_weight=class_weights,
          validation_split=0.1,
          callbacks=[mc])
model.save('model2_V1.h5')
test_loss, test_acc = model.evaluate(X_test, y_test)

print(test_loss, test_acc)

In [ ]:
y_pred = model.predict(X_test, batch_size=2000)
y_pred[y_pred > 0.5] = 1
y_pred[y_pred <= 0.5] = 0
y_pred.flatten()

print('Accuracy:', accuracy_score(y_test, y_pred.flatten()))
print('Precision:', precision_score(y_test, y_pred.flatten()))

In [ ]:
y_pred = model.predict(X_test, batch_size=2000)
y_pred[y_pred > 0.5] = 1
y_pred[y_pred <= 0.5] = 0
y_pred.flatten()

print('Accuracy:', accuracy_score(y_test, y_pred.flatten()))
print('Precision:', precision_score(y_test, y_pred.flatten()))

# Other Transformer

https://keras.io/examples/nlp/text_classification_with_transformer/

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.layers import TransformerBlock

# Prepare the data
input_shape = (100,12)  # input shape of each time series sequence
num_classes = 2  # number of classes for classification


# Define the model
inputs = Input(shape=input_shape)
x = inputs
embedding_dim = 12 
num_heads = 8
ff_dim = 12 

# Time series embedding layer
x = TransformerBlock(embed_dim=embedding_dim, num_heads=num_heads, ff_dim=ff_dim)(x)
x = GlobalMaxPooling1D()(x)
x = Dropout(0.1)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.1)(x)
outputs = Dense(num_classes, activation="softmax")(x)
model = Model(inputs=inputs, outputs=outputs)ss


ssss

# Compile the model
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=1e-4), metrics=["accuracy"])

# Train the model
y_train = tf.keras.utils.to_categorical(y_train)  # convert labels to one-hot encoded vectors
for i in range(0,10):
    model.fit(X_train, y_train, batch_size=32, epochs=1, validation_split=0.1)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
test_acc

In [ ]:
y_pred = model.predict(X_test, batch_size=2000)
y_pred = np.argmax(y_pred, axis=1)

# y_test = tf.keras.utils.to_categorical(y_test)  
print('Accuracy:', accuracy_score(y_test, y_pred.flatten()))
print('Precision:', precision_score(y_test, y_pred.flatten()))